In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# --- INSTALL DEPENDENCIES ---
!pip install sentence-transformers chromadb xarray pandas pyarrow fastparquet --quiet

import os
import pandas as pd
import xarray as xr
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# --- STEP 1: SETUP PATHS ---
drive_base = "/content/drive/MyDrive/ColabNotebooks/SIH2025/Data"
catalog_path = os.path.join(drive_base, "argo_metadata_catalog.csv")
filtered_dir = os.path.join(drive_base, "filtered_argo_data")
os.makedirs(filtered_dir, exist_ok=True)
collection_path = os.path.join(drive_base, "ChromaDB")

# --- STEP 2: LOAD CATALOG ---
catalog = pd.read_csv(catalog_path)
print("📑 Catalog loaded:", catalog.shape)
print(catalog.head())

# --- STEP 3: INITIALIZE EMBEDDING MODEL & CHROMA ---
embedder = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.PersistentClient(path=collection_path)

# Create or get collection
if "argo_collection" in [c.name for c in client.list_collections()]:
    collection = client.get_collection("argo_collection")
else:
    collection = client.create_collection(
        name="argo_collection",
        embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
    )

# --- STEP 4: PREPARE METADATA EMBEDDINGS ---
docs, metas, ids = [], [], []
for idx, row in catalog.iterrows():
    text = (
        f"File {row['file_path']} contains {row.get('num_rows', 'unknown')} measurements "
        f"from {row['year']}-{row.get('month', '')} "
        f"in region lat[{row['lat_min']}, {row['lat_max']}] "
        f"lon[{row['lon_min']}, {row['lon_max']}] "
        f"with depth range {row['depth_min']}–{row['depth_max']} meters."
    )
    docs.append(text)
    metas.append(row.to_dict())
    ids.append(str(idx))

# Insert into Chroma (skip duplicates)
existing_ids = [d['id'] for d in collection.get()['ids']]
to_add_mask = [i not in existing_ids for i in ids]
collection.add(
    documents=[d for d, m in zip(docs, to_add_mask) if m],
    metadatas=[m for m, msk in zip(metas, to_add_mask) if msk],
    ids=[i for i, msk in zip(ids, to_add_mask) if msk]
)
print(f"✅ Vector store ready with {len(collection.get()['ids'])} entries")

# --- STEP 5: HYBRID RETRIEVAL FUNCTION ---
def hybrid_retrieve(query_text, year=None, lat_min=None, lat_max=None,
                    lon_min=None, lon_max=None, depth_min=None, depth_max=None, top_k=5):
    query_emb = embedder.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_emb, n_results=top_k)

    if len(results['metadatas']) == 0:
        return pd.DataFrame()

    matched_files = pd.DataFrame(results['metadatas'][0])

    # Metadata filtering
    if year is not None:
        matched_files = matched_files[matched_files['year'] == year]
    if lat_min is not None:
        matched_files = matched_files[matched_files['lat_max'] >= lat_min]
    if lat_max is not None:
        matched_files = matched_files[matched_files['lat_min'] <= lat_max]
    if lon_min is not None:
        matched_files = matched_files[matched_files['lon_max'] >= lon_min]
    if lon_max is not None:
        matched_files = matched_files[matched_files['lon_min'] <= lon_max]
    if depth_min is not None:
        matched_files = matched_files[matched_files['depth_max'] >= depth_min]
    if depth_max is not None:
        matched_files = matched_files[matched_files['depth_min'] <= depth_max]

    return matched_files

# --- STEP 6: LAZY FILE LOADER ---
def load_file_lazy(file_path):
    try:
        if file_path.endswith(".nc"):
            ds = xr.open_dataset(file_path, decode_times=False)
            df = ds.to_dataframe().reset_index()
        elif file_path.endswith(".parquet"):
            df = pd.read_parquet(file_path)
        else:
            return pd.DataFrame()
        df["source_file"] = file_path
        return df
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return pd.DataFrame()

# --- STEP 7: MERGE DATA FROM RETRIEVAL ---
def get_data_for_query(query_text, year=None, lat_min=None, lat_max=None,
                       lon_min=None, lon_max=None, depth_min=None, depth_max=None):
    files = hybrid_retrieve(query_text, year, lat_min, lat_max, lon_min, lon_max, depth_min, depth_max)
    if files.empty:
        print("⚠️ No files found for this query")
        return pd.DataFrame()

    all_data = []
    for f in files['file_path']:
        if os.path.exists(f):
            df = load_file_lazy(f)
            if not df.empty:
                all_data.append(df)
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        print("⚠️ No data loaded from matching files")
        return pd.DataFrame()

# --- STEP 8: TEST HYBRID RETRIEVAL ---
query_text = "temperature profiles"
df2022 = get_data_for_query(query_text, year=2022, lat_min=-20, lat_max=20, lon_min=30, lon_max=60, depth_min=0, depth_max=2000)
print("Loaded data shape:", df2022.shape)


📑 Catalog loaded: (36, 10)
                                           file_path  year  month    lat_min  \
0  /content/drive/MyDrive/ColabNotebooks/SIH2025/...  2022      1 -39.869000   
1  /content/drive/MyDrive/ColabNotebooks/SIH2025/...  2022      2 -39.906908   
2  /content/drive/MyDrive/ColabNotebooks/SIH2025/...  2022      3 -39.985000   
3  /content/drive/MyDrive/ColabNotebooks/SIH2025/...  2022      4 -39.929090   
4  /content/drive/MyDrive/ColabNotebooks/SIH2025/...  2022      5 -39.912000   

     lat_max   lon_min    lon_max  depth_min    depth_max  num_rows  
0  25.530760  30.86684  109.95030        0.0  4512.899902   1424143  
1  25.096493  30.17752  109.77480        0.0  4510.209961   1279043  
2  25.044137  31.17850  109.79900       -0.2  4512.849609   1396118  
3  26.758967  30.53059  109.89499       -0.1  4502.609863   1465770  
4  24.786320  31.01284  109.53160        0.0  4529.600098   1367667  


TypeError: string indices must be integers, not 'str'

In [4]:
# --- STEP 2: INTERACTIVE PLOTTING FUNCTION (UPDATED) ---
def plot_variable(df, var="TEMP", depth_col="DEPTH_M", interactive=True, title_suffix=""):
    if df.empty:
        print("⚠️ Dataframe is empty. Cannot plot.")
        return
    if var not in df.columns or depth_col not in df.columns:
        print(f"⚠️ Columns {var} or {depth_col} not found in dataframe")
        return

    # Sample large datasets for plotting
    if len(df) > 50000:
        df = df.sample(n=50000, random_state=42)
        print(f"🔹 Sampled 50,000 rows for plotting")

    if interactive:
        fig = px.scatter(
            df,
            x="LONGITUDE",
            y=depth_col,
            color=var,
            color_continuous_scale="Viridis",
            height=600,
            title=f"{var} vs {depth_col} {title_suffix}"
        )
        fig.update_yaxes(autorange="reversed")  # Depth increases downward
        fig.show()
    else:
        import matplotlib.pyplot as plt
        plt.scatter(df["LONGITUDE"], df[depth_col], c=df[var], cmap="viridis", s=5)
        plt.gca().invert_yaxis()
        plt.xlabel("Longitude")
        plt.ylabel(depth_col)
        plt.title(f"{var} vs {depth_col} {title_suffix}")
        plt.colorbar(label=var)
        plt.show()


# --- STEP 3: EXPORT FUNCTION (UPDATED TO USE SAMPLED DATA) ---
def export_data(df, year, base_dir=filtered_dir, max_rows=50000):
    if df.empty:
        print(f"⚠️ No data to export for year {year}")
        return

    # Sample if too large
    if len(df) > max_rows:
        df = df.sample(n=max_rows, random_state=42)
        print(f"🔹 Sampled {max_rows} rows for export")

    csv_path = os.path.join(base_dir, f"argo_{year}_filtered.csv")
    parquet_path = os.path.join(base_dir, f"argo_{year}_filtered.parquet")
    netcdf_path = os.path.join(base_dir, f"argo_{year}_filtered.nc")

    # CSV & Parquet
    df.to_csv(csv_path, index=False)
    df.to_parquet(parquet_path, index=False)

    # NetCDF export
    try:
        ds = xr.Dataset.from_dataframe(df.set_index(['JULD', 'DEPTH_M', 'LATITUDE', 'LONGITUDE']))
        ds.to_netcdf(netcdf_path)
        print(f"✅ Exported CSV, Parquet, NetCDF for year {year}")
    except Exception as e:
        print(f"⚠️ NetCDF export failed: {e}")


# --- STEP 4: LOAD DATA (USING SUMMARIZED OR PART OF DATA) ---
query_text = "temperature profiles"
df2022 = get_data_for_query(
    query_text,
    year=2022,
    lat_min=-20,
    lat_max=20,
    lon_min=30,
    lon_max=60,
    depth_min=0,
    depth_max=2000
)

# --- STEP 5: PLOT INTERACTIVE VARIABLE ---
plot_variable(df2022, var="TEMP", depth_col="DEPTH_M", interactive=True, title_suffix="2022")

# --- STEP 6: EXPORT DATA ---
export_data(df2022, year=2022)


🔹 Sampled 50,000 rows for plotting


🔹 Sampled 50000 rows for export
⚠️ NetCDF export failed: cannot convert a DataFrame with a non-unique MultiIndex into xarray


In [6]:
# --- INSTALL DEPENDENCIES ---
!pip install plotly xarray pandas pyarrow fastparquet --quiet

import os
import pandas as pd
import xarray as xr
import plotly.express as px

# --- STEP 1: SETUP PATHS ---
filtered_dir = "/content/drive/MyDrive/ColabNotebooks/SIH2025/Data/filtered_argo_data"
summary_dir = os.path.join(filtered_dir, "summaries")
plot_dir = os.path.join(filtered_dir, "plots")
os.makedirs(summary_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

# --- STEP 2: FUNCTION TO SAMPLE DATA ---
def sample_df(df, fraction=0.1, max_rows=5000):
    if df.empty:
        return df
    n = min(len(df), max_rows)
    return df.sample(n=n, random_state=42)

# --- STEP 3: FUNCTION TO PLOT VARIABLE ---
def plot_variable(df, var="TEMP", depth_col="DEPTH_M", year=None, interactive=True):
    if df.empty or var not in df.columns or depth_col not in df.columns:
        print(f"⚠️ Columns {var} or {depth_col} missing for year {year}")
        return

    title = f"{var} vs {depth_col} ({year})"
    if interactive:
        fig = px.scatter(df, x="LONGITUDE", y=depth_col, color=var,
                         color_continuous_scale="Viridis", height=600, title=title)
        fig.update_yaxes(autorange="reversed")
        fig.write_html(os.path.join(plot_dir, f"{var}_{year}.html"))
        fig.show()
    else:
        import matplotlib.pyplot as plt
        plt.scatter(df["LONGITUDE"], df[depth_col], c=df[var], cmap="viridis", s=2)
        plt.gca().invert_yaxis()
        plt.xlabel("Longitude")
        plt.ylabel(depth_col)
        plt.title(title)
        plt.colorbar(label=var)
        plt.savefig(os.path.join(plot_dir, f"{var}_{year}.png"), dpi=150)
        plt.close()

# --- STEP 4: FUNCTION TO COMPUTE SUMMARY STATS ---
def summarize_by_depth(df, var_list=["TEMP", "PSAL"], depth_col="DEPTH_M", year=None):
    if df.empty:
        return pd.DataFrame()
    summary = df.groupby(depth_col)[var_list].agg(['mean', 'min', 'max']).reset_index()
    summary_path = os.path.join(summary_dir, f"summary_{year}.csv")
    summary.to_csv(summary_path, index=False)
    print(f"✅ Summary stats saved: {summary_path}")
    return summary

# --- STEP 5: LOOP THROUGH YEARS ---
years = [2022, 2023, 2024]

for year in years:
    print(f"\n🔹 Processing year {year} ...")
    # Load filtered data
    file_path = os.path.join(filtered_dir, f"argo_{year}_filtered.parquet")
    if not os.path.exists(file_path):
        print(f"⚠️ File not found: {file_path}")
        continue

    df = pd.read_parquet(file_path)

    # Sample for faster plotting
    df_sample = sample_df(df, max_rows=5000)

    # Plot TEMP and PSAL
    plot_variable(df_sample, var="TEMP", depth_col="DEPTH_M", year=year, interactive=True)
    plot_variable(df_sample, var="PSAL", depth_col="DEPTH_M", year=year, interactive=True)

    # Summary stats
    summarize_by_depth(df, var_list=["TEMP", "PSAL"], depth_col="DEPTH_M", year=year)



🔹 Processing year 2022 ...


✅ Summary stats saved: /content/drive/MyDrive/ColabNotebooks/SIH2025/Data/filtered_argo_data/summaries/summary_2022.csv

🔹 Processing year 2023 ...


✅ Summary stats saved: /content/drive/MyDrive/ColabNotebooks/SIH2025/Data/filtered_argo_data/summaries/summary_2023.csv

🔹 Processing year 2024 ...


✅ Summary stats saved: /content/drive/MyDrive/ColabNotebooks/SIH2025/Data/filtered_argo_data/summaries/summary_2024.csv


In [2]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import re
import os
import xarray as xr

# -----------------------
# --- HELPER FUNCTIONS ---
# -----------------------

def parse_query(query):
    """
    Extract variable, year, lat/lon/depth ranges from query text.
    Fallbacks are None if not found.
    """
    query = query.lower()

    # Variable detection
    if "salinity" in query or "psal" in query:
        var = "PSAL"
    elif "temperature" in query or "temp" in query:
        var = "TEMP"
    else:
        var = "TEMP"  # default

    # Year detection
    year_match = re.search(r"(19|20)\d{2}", query)
    year = int(year_match.group()) if year_match else None

    # Lat/Lon/Depth ranges (default None = full range)
    lat_min = lat_max = lon_min = lon_max = depth_min = depth_max = None

    return var, year, lat_min, lat_max, lon_min, lon_max, depth_min, depth_max


def plot_variable(df, var="TEMP", depth_col="DEPTH_M", interactive=True):
    if var not in df.columns or depth_col not in df.columns:
        print(f"⚠️ Variable {var} or depth column {depth_col} not found.")
        return

    if interactive:
        fig = px.scatter(
            df, x=var, y=depth_col, color=var,
            color_continuous_scale="Viridis",
            title=f"{var} vs Depth",
            height=600
        )
        fig.update_yaxes(autorange="reversed")  # Depth increases downward
        fig.show()
    else:
        df.plot(x=var, y=depth_col, kind="scatter")
        plt.gca().invert_yaxis()
        plt.title(f"{var} vs Depth")
        plt.show()


def generate_natural_language_insights(
    df, variables=["TEMP", "PSAL"],
    depth_col="DEPTH_M", lat_col="LATITUDE", lon_col="LONGITUDE"
):
    if df.empty:
        return ["⚠️ No data available for insights."]

    insights = []

    for var in variables:
        if var not in df.columns:
            continue

        val_mean = df[var].mean()
        val_min = df[var].min()
        val_max = df[var].max()

        insights.append(
            f"The average {var.lower()} is {val_mean:.2f}, ranging from {val_min:.2f} to {val_max:.2f}."
        )

        if depth_col in df.columns:
            corr = np.corrcoef(df[depth_col], df[var])[0,1]
            if corr > 0.1:
                insights.append(f"{var} tends to increase with depth (corr={corr:.2f}).")
            elif corr < -0.1:
                insights.append(f"{var} tends to decrease with depth (corr={corr:.2f}).")
            else:
                insights.append(f"No strong trend of {var} with depth (corr={corr:.2f}).")

        if lat_col in df.columns and lon_col in df.columns:
            max_row = df.loc[df[var].idxmax()]
            min_row = df.loc[df[var].idxmin()]
            insights.append(
                f"Highest {var.lower()} = {max_row[var]:.2f} at (lat={max_row[lat_col]:.2f}, lon={max_row[lon_col]:.2f}, depth={max_row[depth_col]:.2f}m)."
            )
            insights.append(
                f"Lowest {var.lower()} = {min_row[var]:.2f} at (lat={min_row[lat_col]:.2f}, lon={min_row[lon_col]:.2f}, depth={min_row[depth_col]:.2f}m)."
            )

    insights.append(
        f"Coverage: lat {df[lat_col].min():.2f}–{df[lat_col].max():.2f}, "
        f"lon {df[lon_col].min():.2f}–{df[lon_col].max():.2f}, "
        f"depth {df[depth_col].min():.2f}–{df[depth_col].max():.2f}m."
    )

    random.shuffle(insights)
    return insights


# -----------------------
# --- CONVERSATIONAL LOOP ---
# -----------------------

def conversational_loop_hybrid():
    print("🌊 Argo Data Assistant (Hybrid Retrieval + NL Insights) ready! Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")
        if user_input.strip().lower() == "exit":
            print("👋 Exiting Argo Data Assistant.")
            break

        # Parse query
        var, year, lat_min, lat_max, lon_min, lon_max, depth_min, depth_max = parse_query(user_input)

        print(f"\n🔹 Processing query: {user_input}")

        # Retrieve data using your hybrid pipeline
        df = get_data_for_query(
            query_text=user_input,
            year=year,
            lat_min=lat_min,
            lat_max=lat_max,
            lon_min=lon_min,
            lon_max=lon_max,
            depth_min=depth_min,
            depth_max=depth_max
        )

        if not df.empty:
            # Sample for faster plotting
            df_sample = df.sample(min(5000, len(df)), random_state=42)

            # Plot
            plot_variable(df_sample, var=var, depth_col="DEPTH_M", interactive=True)

            # Generate insights
            nl_insights = generate_natural_language_insights(df, variables=[var])
            print("\n📝 Natural Language Insights:")
            for line in nl_insights:
                print(line)
            print("\n✅ Query processed. Enter another query or 'exit'.\n")
        else:
            print("⚠️ No data available for this query.\n")

# --- RUN ---
conversational_loop_hybrid()


🌊 Argo Data Assistant (Hybrid Retrieval + NL Insights) ready! Type 'exit' to quit.

You: temp

🔹 Processing query: temp


NameError: name 'get_data_for_query' is not defined